In [19]:
import os
import cv2
import glob
from glob import glob
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot
import matplotlib.pyplot
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from multiprocessing import Pool, cpu_count
from PIL import ImageFilter, ImageStat, Image, ImageDraw
from IPython.display import display
import keras

In [7]:
def show(X):
    display(Image.fromarray(X))
    
def im_multi(path):
    try:
        #print(path)
        im_stats_im_ = cv2.imread(path)
        #print(im_stats_im_.size)
        return [path, {'size': im_stats_im_.size}]
    except:
        print("Error path:%s"%(path))
        return [path, {'size': [0, 0]}]
    
def im_stats(im_stats_df):
    im_stats_d = {}
    p = Pool(cpu_count())
    ret = p.map(im_multi, im_stats_df['path'])
    for i in range(len(ret)):
        im_stats_d[ret[i][0]] = ret[i][1]
    im_stats_df['size'] = im_stats_df['path'].map(lambda x: ' '.join([str(s) for s in str(im_stats_d[x]['size'])]))
    return im_stats_df

In [8]:
import glob
train1 = glob.glob('/home/dataset/train/**/*.jpg')
train1 =\
pd.DataFrame([[p.split('/')[4],p.split('/')[5],p] for p in train1], columns = ['type','image','path'])

train2 = glob.glob('/home/dataset/Type_*/*.jpg')
train2 =\
pd.DataFrame([[p.split('/')[3],p.split('/')[4],p] for p in train2], columns = ['type','image','path'])

train = pd.concat([train1, train2]).reset_index(drop=True)
train.head()

,type,image,path
0,Type_3,477.jpg,/home/dataset/train/Type_3/477.jpg
1,Type_3,1153.jpg,/home/dataset/train/Type_3/1153.jpg
2,Type_3,335.jpg,/home/dataset/train/Type_3/335.jpg
3,Type_3,324.jpg,/home/dataset/train/Type_3/324.jpg
4,Type_3,98.jpg,/home/dataset/train/Type_3/98.jpg


In [9]:
test = glob.glob('/home/dataset/test/*.jpg')
test = pd.DataFrame([[p.split('/')[4],p] for p in test], columns = ['image','path'])

test_id = test.image.values
test.head()

,image,path
0,477.jpg,/home/dataset/test/477.jpg
1,178.jpg,/home/dataset/test/178.jpg
2,335.jpg,/home/dataset/test/335.jpg
3,324.jpg,/home/dataset/test/324.jpg
4,98.jpg,/home/dataset/test/98.jpg


In [10]:
test_id = test.image.values

In [11]:
train = pd.read_csv('TRAIN.csv')

In [12]:
test = pd.read_csv('TEST.csv')

In [13]:
le = LabelEncoder()
train_target = le.fit_transform(train['type'].values)
print(le.classes_)

['Type_1' 'Type_2' 'Type_3']


In [29]:
# show(cv2.imread(not_cervix[0]))

In [30]:
# cv2.imread(not_cervix[0])

In [44]:
def get_im_cv2(args):
    
    img = cv2.imread(args[0])
    #print('start gray')
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, threshold = cv2.threshold(img_gray, 120, 255, cv2.THRESH_BINARY)
    im2, contours, hierarchy = cv2.findContours(threshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    rects = [cv2.boundingRect(cnt) for cnt in contours]
    
    try: 
        top_x = min([x for (x, y, w, h) in rects])
        top_y = min([y for (x, y, w, h) in rects])
        bottom_x = max([x+w for (x, y, w, h) in rects])
        bottom_y = max([y+h for (x, y, w, h) in rects])  
    except:
        print('Error1 image: %s'%(args[0]))
    
    try:
        crop = img[top_y:bottom_y, top_x:bottom_x]
        resized = cv2.resize(crop, (args[1], args[1]), cv2.INTER_LINEAR)
        return [args[0], resized]
    except:
        print('Error2 image: %s'%(args[0]))
        crop = img
        resized = cv2.resize(crop, (args[1], args[1]), cv2.INTER_LINEAR)
        return [args[0], resized]

def image_features(paths, dim):
    
    imf_d = {}
    p = Pool(cpu_count())
    dims = [dim for i in range(len(paths.tolist()))]
    params = list(zip(paths.tolist(), dims))
    ret = p.map(get_im_cv2, params)
    #print(ret)
    for i in range(len(ret)):
        imf_d[ret[i][0]] = ret[i][1]
    ret = []
    fdata = [imf_d[f] for f in paths]
    fdata = np.array(fdata, dtype = np.uint8)
    
    return fdata

In [45]:
dim = 64
train_data = image_features(train['path'], dim)

Error1 image: /home/dataset/Type_3/6240.jpg
Error2 image: /home/dataset/Type_3/6240.jpg
Error1 image: /home/dataset/Type_1/2239.jpg
Error2 image: /home/dataset/Type_1/2239.jpg
Error1 image: /home/dataset/Type_1/3433.jpg
Error2 image: /home/dataset/Type_1/3433.jpg
Error1 image: /home/dataset/Type_1/5948.jpg
Error2 image: /home/dataset/Type_1/5948.jpg
Error1 image: /home/dataset/Type_3/6230.jpg
Error2 image: /home/dataset/Type_3/6230.jpg
Error1 image: /home/dataset/Type_1/382.jpg
Error2 image: /home/dataset/Type_1/382.jpg
Error1 image: /home/dataset/Type_1/6028.jpg
Error2 image: /home/dataset/Type_1/6028.jpg
Error1 image: /home/dataset/Type_1/2391.jpg
Error2 image: /home/dataset/Type_1/2391.jpg
Error1 image: /home/dataset/Type_1/563.jpg
Error2 image: /home/dataset/Type_1/563.jpg
Error1 image: /home/dataset/Type_1/1941.jpg
Error2 image: /home/dataset/Type_1/1941.jpg
Error1 image: /home/dataset/Type_1/3611.jpg
Error2 image: /home/dataset/Type_1/3611.jpg
Error1 image: /home/dataset/Type_2/2

In [47]:
train_data.shape

(8212, 64, 64, 3)

In [75]:
# %timeit np.save('ALL_TRAIN_SEGCV_64.npy', train_data[:,:,:,::-1])

In [77]:
# train_data = np.load('ALL_TRAIN_SEGCV_64.npy').shape

In [ ]:
test_data = image_features(test['path'], dim)

In [ ]:
%timeit np.save('ALL_TEST_SEGCV_64.npy', test_data[:,:,:,::-1])

In [ ]:
test_data

In [24]:
not_cervix =\
[
    '/home/dataset/Type_1/746.jpg',
    '/home/dataset/Type_1/2030.jpg',
    '/home/dataset/Type_1/4065.jpg',
    '/home/dataset/Type_1/4702.jpg',
    '/home/dataset/Type_1/4706.jpg',
    '/home/dataset/Type_2/1813.jpg',
    '/home/dataset/Type_2/3086.jpg',
]